In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\
import matplotlib.pyplot as plt
from PIL import Image, ImageChops
import cv2
import numpy as np

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from glob import glob
basepath = '../input/train/'

all_cervix_images = []

for path in sorted(glob(basepath + "*")):
    cervix_type = path.split("/")[-1]
    cervix_images = sorted(glob(basepath + cervix_type + "/*"))
    all_cervix_images = all_cervix_images + cervix_images

all_cervix_images = pd.DataFrame({'imagepath': all_cervix_images})
all_cervix_images['filetype'] = all_cervix_images.apply(lambda row: row.imagepath.split(".")[-1], axis=1)
all_cervix_images['type'] = all_cervix_images.apply(lambda row: row.imagepath.split("/")[-2], axis=1)
all_cervix_images.head()

In [ ]:
def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

In [ ]:
fig = plt.figure(figsize=(12,8))

i = 1
for t in all_cervix_images['type'].unique():
    ax = fig.add_subplot(1,3,i)
    f = all_cervix_images[all_cervix_images['type'] == t]['imagepath'].values[0]
    #plt.imshow(plt.imread(f))
    #plt.title('sample for cervix {}'.format(t))
    
    im = Image.open(f).convert("L")
    im = trim(im)
    arr = np.asarray(im)
    #if (i==2):
        #print(arr.tolist())
    plt.imshow(arr, cmap='gray')
    #plt.title('sample for cervix {}'.format(t))
    
    i+=1

In [ ]:
import sys
from scipy.misc import imread
from scipy.misc import imshow
from scipy import sum, average

In [ ]:
def to_grayscale(arr):
    "If arr is a color image (3D array), convert it to grayscale (2D array)."
    if len(arr.shape) == 3:
        return average(arr, -1)  # average over the last axis (color channels)
    else:
        return arr

In [ ]:
data = []
target = []
feature_names = ['image_array']

numpy_array = np.empty()

for t in all_cervix_images['type'].unique():
    i = 1
    for i in range(1):
        image_name = all_cervix_images[all_cervix_images['type'] == t]['imagepath'].values[i]
        image_array = to_grayscale(imread(image_name).astype(float))
        data.append(image_array)
        target.append(t)
        
        print(imread(image_name))

print(data)
#df = pd.DataFrame(data)
#df.head()

In [ ]:
print(img1.min())
print(img2.min())
print(img3.min())

print(img1.max())
print(img2.max())
print(img3.max())

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
print(iris)
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()
len(digits.images[4])

In [ ]:
import cv2
import math
from sklearn import mixture
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.misc import imread
from scipy.misc import imshow
from scipy import sum, average
from skimage import feature
from skimage.transform import resize
from sklearn import datasets, svm, metrics
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              AdaBoostClassifier)

from glob import glob
basepath = '../input/train/'

In [ ]:
def to_grayscale(arr):
    "If arr is a color image (3D array), convert it to grayscale (2D array)."
    if len(arr.shape) == 3:
        return average(arr, -1)  # average over the last axis (color channels)
    else:
        return arr

In [ ]:
all_cervix_images = []

for path in sorted(glob(basepath + "*")):
    cervix_type = path.split("/")[-1]
    cervix_images = sorted(glob(basepath + cervix_type + "/*"))
    all_cervix_images = all_cervix_images + cervix_images

all_cervix_images = pd.DataFrame({'imagepath': all_cervix_images})
all_cervix_images['filetype'] = all_cervix_images.apply(lambda row: row.imagepath.split(".")[-1], axis=1)
all_cervix_images['type'] = all_cervix_images.apply(lambda row: row.imagepath.split("/")[-2], axis=1)

In [ ]:
train_data = []
train_target = []
test_data = []
test_target = []
raw_data = []
feature_names = ['image_array']

all_samples = 10
train_samples = 7

for t in all_cervix_images['type'].unique():
    i = 1
    for i in range(all_samples):
        image_name = all_cervix_images[all_cervix_images['type'] == t]['imagepath'].values[i]
        image_array = resize(to_grayscale(imread(image_name).astype(float)), (200, 200))
        sample=imread(image_name)
        raw_data.append(imread(image_name))
        if i > train_samples:
            test_data.append(image_array.flatten())
            test_target.append(t)
        else:
            train_data.append(image_array.flatten())
            train_target.append(t)

print(train_target)
print(test_target)
print(len(train_data))
print(len(test_data))

In [ ]:
frame = raw_data[0]

converted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

lower = np.array([0, 48, 80], dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")
skinMask = cv2.inRange(converted, lower, upper)
 
# apply a series of erosions and dilations to the mask
# using an elliptical kernel
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
skinMask = cv2.erode(skinMask, kernel, iterations = 2)
skinMask = cv2.dilate(skinMask, kernel, iterations = 2)

skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
skin = cv2.bitwise_and(frame, frame, mask = skinMask)

cv2.imshow("images", np.hstack([frame, skin]))

In [ ]:
def Ra_space(img, Ra_ratio, a_threshold):
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = np.zeros((w*h, 2))
    for i in range(w):
        for j in range(h):
            R = math.sqrt((w/2-i)*(w/2-i) + (h/2-j)*(h/2-j))
            Ra[i*h+j, 0] = R
            Ra[i*h+j, 1] = min(imgLab[i][j][1], a_threshold)
            
    Ra[:,0] /= max(Ra[:,0])
    Ra[:,0] *= Ra_ratio
    Ra[:,1] /= max(Ra[:,1])

    return Ra

In [ ]:
mask_color = [0, 0, 0]
image = raw_data[4]

# a channel saturation threshold
a_threshold = 300

# creating the R-a feature for the image
Ra_array = Ra_space(image, 1.0, a_threshold)

# k-means gaussian mixture model
g = mixture.GaussianMixture(n_components = 2, covariance_type = 'diag', random_state = 0, init_params = 'kmeans')
image_array_sample = shuffle(Ra_array, random_state=0)[:1000]
g.fit(image_array_sample)
labels = g.predict(Ra_array)

# creating the mask array and assign the correct cluster label
boolean_image_mask = np.array(labels).reshape(image.shape[0], image.shape[1])
outer_cluster_label = boolean_image_mask[0,0]

new_image = image.copy()

for i in range(boolean_image_mask.shape[0]):
    for j in range(boolean_image_mask.shape[1]):
        if boolean_image_mask[i, j] == outer_cluster_label:
            new_image[i, j] = mask_color

a = np.hstack([image, new_image])

plt.imshow(a)